In [2]:
import pandas as pd
from pathlib import Path
import re

playlist_dir = Path("Playlists")
csv_files = list(playlist_dir.rglob("*.csv"))

dfs = []
for path in csv_files:
    try:
        df = pd.read_csv(path)
    except Exception as e:
        print(f"Skipping {path}: {e}")
        continue

    # User is the first subfolder under Playlist (e.g., Playlist/<User>/file.csv)
    try:
        rel = path.parent.relative_to(playlist_dir)
        user = rel.parts[0] if rel.parts else ""
    except Exception:
        user = ""

    # Extract a 4-digit year (e.g., 2023) from the filename, if present
    m = re.search(r'(?<!\d)(19|20)\d{2}(?!\d)', path.name)
    playlist_year = int(m.group(0)) if m else pd.NA

    df["User"] = user
    df["playlist year"] = playlist_year
    dfs.append(df)

if dfs:
    df_playlists = pd.concat(dfs, ignore_index=True, sort=False)
else:
    df_playlists = pd.DataFrame()

# df_playlists now contains all CSVs with "User" and "playlist year" columns
df_playlists

,Track URI,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Explicit,Added By,Added At,...,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,User,playlist year
0,spotify:track:2D2lazsae9o1UoVPUAdxyT,21,21,Gracie Abrams,2020-02-20,185172,31,False,NaN,1970-01-01T00:00:00Z,...,1,0.0899,0.516000,0.000000,0.1090,0.4020,125.055,4,Emily,2023
1,spotify:track:2RvLlxea9woaqb2ZLgAmv7,emma,emma,Casper,2023-06-15,195066,40,False,NaN,1970-01-01T00:00:00Z,...,1,0.0429,0.669000,0.000073,0.2870,0.2640,163.857,4,Emily,2023
2,spotify:track:53dtP2iUMvaF28JZcHnFuU,logical,GUTS,Olivia Rodrigo,2023-09-08,231907,71,False,NaN,1970-01-01T00:00:00Z,...,1,0.0323,0.844000,0.000000,0.0990,0.1520,80.686,4,Emily,2023
3,spotify:track:401qSkk4KOeBVld2aiCAoK,Lovers to Strangers,Lovers to Strangers,Chance Peña,2023-07-14,174086,52,True,NaN,1970-01-01T00:00:00Z,...,1,0.0312,0.098700,0.000000,0.3440,0.2020,116.092,4,Emily,2023
4,spotify:track:2hkcv7xQJgmLvCY3XSixjG,Dussmann,OLYMPIA,Betterov,2022-10-14,217551,45,False,NaN,1970-01-01T00:00:00Z,...,1,0.0563,0.000053,0.072400,0.1430,0.6960,89.978,4,Emily,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,spotify:track:2jdAk8ATWIL3dwT47XpRfu,Slow Dancing in a Burning Room,Continuum,John Mayer,2006-09-11,241946,78,False,NaN,1970-01-01T00:00:00Z,...,0,0.0273,0.575000,0.005060,0.0805,0.2900,134.018,5,William,2025
1196,spotify:track:4QlzkaRHtU8gAdwqjWmO8n,Friday I'm In Love,Wish,The Cure,1992-01-01,215160,79,False,NaN,1970-01-01T00:00:00Z,...,0,0.0341,0.001260,0.000068,0.3590,0.5140,136.205,4,William,2025
1197,spotify:track:3wlNmUeoHnM1jcOcXxC6qI,Rivendell,The Lord of the Rings: The Fellowship of the R...,Howard Shore,2001,206026,56,False,NaN,1970-01-01T00:00:00Z,...,1,0.0439,0.972000,0.912000,0.1050,0.0364,86.930,4,William,2025
1198,spotify:track:3p428gAKm9dBoMjBVOIvVu,I Can't Hear It Now (from the series Arcane Le...,Arcane League of Legends: Season 2 (Soundtrack...,Arcane;Freya Ridings,2024-11-23,161040,64,False,NaN,1970-01-01T00:00:00Z,...,0,0.0319,0.819000,0.000085,0.0756,0.1550,119.398,4,William,2025


In [3]:
# tracks that appear for more than one unique user based on Track Name + Artist
multi_user_counts = df_playlists.groupby(['Track Name', 'Artist Name(s)'])['User'].nunique()
multi_user_tracks = multi_user_counts[multi_user_counts > 1].sort_values(ascending=False)

print(f"Tracks (by name+artist) present for more than one user: {len(multi_user_tracks)}")
print(multi_user_tracks.head(20))

# detailed summary: Track Name, Artist(s), list of users, number of users, total occurrences, distinct URIs
multi_user_summary = (
    df_playlists
    .groupby(['Track Name', 'Artist Name(s)'])
    .agg(users=('User', lambda x: sorted(x.unique())),
         user_count=('User', 'nunique'),
         total_count=('User', 'size'),
         track_uris=('Track URI', lambda x: sorted(x.unique())))
    .reset_index()
    .query('user_count > 1')
    .sort_values(['user_count', 'total_count'], ascending=[False, False])
)

multi_user_summary

Tracks (by name+artist) present for more than one user: 10
Track Name                                               Artist Name(s) 
Baller                                                   Abor & Tynna       2
Cha Cha Cha                                              Käärijä            2
Good Luck, Babe!                                         Chappell Roan      2
I Love You, I'm Sorry                                    Gracie Abrams      2
Mona Lisa                                                Abor & Tynna       2
Sarà perché ti amo                                       Ricchi E Poveri    2
That’s So True                                           Gracie Abrams      2
The A Team                                               Ed Sheeran         2
What Was I Made For? [From The Motion Picture "Barbie"]  Billie Eilish      2
i like the way you kiss me                               Artemas            2
Name: User, dtype: int64


,Track Name,Artist Name(s),users,user_count,total_count,track_uris
137,Cha Cha Cha,Käärijä,"[Emily, William]",2,3,[spotify:track:5RX8T3EoTuXcybAxe6oPAw]
864,i like the way you kiss me,Artemas,"[Emily, William]",2,3,[spotify:track:2GxrNKugF82CnoRFbQfzPf]
81,Baller,Abor & Tynna,"[Emily, William]",2,2,"[spotify:track:08HAsedFOMxs7NETNXPZ3K, spotify..."
322,"Good Luck, Babe!",Chappell Roan,"[Emily, William]",2,2,[spotify:track:0WbMK4wrZ1wFSty9F7FCgu]
367,"I Love You, I'm Sorry",Gracie Abrams,"[Emily, William]",2,2,[spotify:track:51rfRCiUSvxXlCSCfIztBy]
492,Mona Lisa,Abor & Tynna,"[Emily, William]",2,2,"[spotify:track:0GuxW4nOIvbZ2cIrdJX85C, spotify..."
629,Sarà perché ti amo,Ricchi E Poveri,"[Emily, William]",2,2,"[spotify:track:6OMjtuz1sYVsHpdeZPUR1n, spotify..."
710,That’s So True,Gracie Abrams,"[Emily, William]",2,2,[spotify:track:7ne4VBA60CxGM75vw0EYad]
711,The A Team,Ed Sheeran,"[Emily, William]",2,2,[spotify:track:1VdZ0vKfR5jneCmWIUAMxK]
789,What Was I Made For? [From The Motion Picture ...,Billie Eilish,"[Emily, William]",2,2,[spotify:track:6wf7Yu7cxBSPrRlWeSeK0Q]


In [17]:
# find the oldest song(s) in df_playlists by Release Date, showing only the year
release_dt = pd.to_datetime(df_playlists['Release Date'], errors='coerce', format='mixed')
release_year = release_dt.dt.year.astype('Int64')  # nullable integer year

df_with_dt = df_playlists.assign(release_dt=release_dt, release_year=release_year)

if df_with_dt['release_year'].notna().any():
    min_year = int(df_with_dt['release_year'].min())
    oldest_songs = df_with_dt[df_with_dt['release_year'] == min_year][
        ['Track Name', 'Artist Name(s)', 'Track URI', 'Release Date', 'release_year', 'User', 'playlist year']
    ].reset_index(drop=True)
    print(f"Oldest release year: {min_year}")
    oldest_songs
else:
    print("No valid release years found.")

Oldest release year: 1975


In [18]:
df_with_dt = df_with_dt[["Track URI", "Track Name", "Artist Name(s)", "Duration (ms)" , "Popularity", "Explicit", "User", "playlist year", "release_year"]]
df_with_dt
df_with_dt.to_csv("filtered_list.csv", index=False)